### Environment test

In [1]:
import os

#Compare UID and GID from Operating System and notebook, make sure they match
real_uid = os.getuid()
notebook_uid = !id -u

real_gid = os.getgid()
notebook_gid = !id -g

if real_uid != int(notebook_uid[0]) or real_gid != int(notebook_gid[0]):
    print("No match")
    print(real_uid, notebook_uid, real_gid, notebook_gid)
else:
    print("Match")

Match


In [ ]:
#Load environment variables from system into python

OPE_UID = os.environ['OPE_UID']
OPE_GID = os.environ['OPE_GID']
OPE_GROUP = os.environ['OPE_GROUP']
NB_GROUP = os.environ['NB_GROUP'] 
NB_USER = os.environ['NB_USER']
XDG_CACHE_HOME = f"/home/{NB_USER}/.cache/"
#Verify environment variables are correct
notebook_uid = !id -u
notebook_gid = !id -g

if int(notebook_uid[0]) != int(OPE_UID) or int(notebook_gid[0]) != int(OPE_GID):
    raise ValueError("Notebook UID or GID does not meet expected values.")

    
if OPE_GROUP != 'root' or NB_GROUP != 'root':
    raise ValueError("OPE_GROUP Or NB_GROUP does not match 'root'.")

if NB_USER != 'jovyan':
    raise ValueError("NB_USER does not match 'jovyan'.")

if XDG_CACHE_HOME != '/home/jovyan/.cache/':
    raise ValueError("XDD_CACHE_HOME does not match expected path.")

In [ ]:
%%bash
# ASLR test: run gdb 100 times on date binary stopping at first instruction and grep ld. If ld is found at same address then ASLR
# is working properly
count=$(for ((i=0;i<100; i++)); do gdb -ex starti -ex quit -q --batch /usr/bin/date 2>/dev/null | grep ld; done | tee out | uniq | wc -l)
if [[ $count == 1 ]]; then
   echo "ASLR test: OK: ASLR properly starts binary with a constant instruction address"
else
   echo "ASLR test: ERROR: Failed to start a binary with a constant instruction address: $count"
fi

# from https://docs.oracle.com/en/operating-systems/oracle-linux/6/security/ol_clp_sec.html:
# "kernel.randomize_va_space controls Address Space Layout Randomization (ASLR), which can help defeat certain types of buffer overflow 
# attacks. A value of 0 disables ASLR, 1 randomizes the positions of the stack, virtual dynamic shared object (VDSO) page, and shared memory 
# regions, and 2 randomizes the positions of the stack, VDSO page, shared memory regions, and the data segment. The default and recommended 
# setting is 2."

aslr_status=$(cat /proc/sys/kernel/randomize_va_space)
if [[ $aslr_status == 2 ]]; then
    echo "ASLR test: OK: Full ASLR is enabled"
elif [[ $aslr_status == 1 ]]; then
    echo "ASLR test: OK: Partial ASLR is enabled"
else
    echo "ASLR test: ERROR: ASLR is disabled"
fi

In [ ]:
%%bash
# Ping test to check internet connectivity
ping -c 3 google.com > /dev/null 2>&1
# if zero exit code from above command, ping is successful
if [[ $? == 0 ]]; then
    echo "Ping test: OK: Internet connection is working."
else
    echo "Ping test: ERROR: Internet connection is not working."
fi

### Write Permission to Home Directory Test

In [ ]:
import subprocess
import os

def shelltest(CMD):
    try:
        result = subprocess.check_output(CMD, shell=True, stderr=subprocess.STDOUT)
        return 0, result.decode('utf-8')
    except subprocess.CalledProcessError as e:
        return e.returncode, e.output.decode('utf-8')
    except Exception as e:  # new add: handle exceptions 
        return -1, str(e)

# Test to check write permissions to home directory
TEST = "WRITE PERMISSION TO HOME DIRECTORY"
CMD = f"touch {os.path.expanduser('~')}/test_write_permissions.tmp && echo 'Write Permission: Yes' && rm {os.path.expanduser('~')}/test_write_permissions.tmp || echo 'Write Permission: No'"

# Execute Test
ERRORS, output = shelltest(CMD)
print(f"Output of {TEST}:\n{output}")

In [ ]:
if 'Write Permission: No' in output:
    ERRORS += 1
if ERRORS > 0:
    print("THERE ARE " + str(ERRORS)  + " ERRORS")
else:
    print("ALL TESTS PASS")
print("ERRORS:", ERRORS)

### Pip-Conda Test

In [ ]:
import subprocess
import os
import sys
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

def shelltest(CMD):
    """Executes a shell command and returns the result."""
    try:
        result = subprocess.check_output(CMD, shell=True, stderr=subprocess.STDOUT)
        return 0, result.decode('utf-8')
    except subprocess.CalledProcessError as e:
        logger.error("Shell command failed with CalledProcessError: %s", str(e))
        return e.returncode, e.output.decode('utf-8')
    except Exception as e:
        logger.exception("Unexpected error when running shell command.")
        return -1, str(e)

ERRORS = 0

In [ ]:
TEST = "PIP PACKAGE INSTALLATION"
CMD = "pip install --user pytest || echo 'PIP Install: No'"
ret_code, output = shelltest(CMD)
logger.info(f"Output of {TEST}:\n{output}")

if 'PIP Install: No' in output or ret_code != 0:
    ERRORS += 1
    logger.error("PIP package installation failed.")

### Test if conda directories are read/writable

In [ ]:
def check_permissions(dir_path):
    """Check if a directory is readable and writable."""
    try:
        readable = os.access(dir_path, os.R_OK)
        writable = os.access(dir_path, os.W_OK)
        return readable, writable
    except Exception as e:
        logger.exception(f"Error checking permissions for {dir_path}.")
        return False, False

# Identify the conda directories
conda_base_dir = os.path.abspath(os.path.join(os.path.dirname(os.sys.executable), ".."))
conda_env_dir = os.environ.get('CONDA_PREFIX', '')

unaccessible_dirs = 0

# Ensure conda directories are available
if not (conda_base_dir or conda_env_dir):
    logger.error("Conda directory not found. Skipping test.")
    ERRORS += 1

for dir_name, dir_path in [('Conda Base Directory', conda_base_dir), 
                           ('Conda Environment Directory', conda_env_dir)]:
    readable, writable = check_permissions(dir_path)
    logger.info(f"{dir_name}: {dir_path}")
    logger.info(f"Readable: {readable}")
    logger.info(f"Writable: {writable}\n")

    if not (readable and writable):
        unaccessible_dirs += 1

if unaccessible_dirs == 2:
    ERRORS += 1
    logger.error("Both Conda directories are not fully accessible.")

In [ ]:
if ERRORS > 0:
    logging.error(f"THERE ARE {ERRORS} ERRORS")
else:
    logging.info("ALL TESTS PASS")

logging.info(f"ERRORS: {ERRORS}")